<a href="https://colab.research.google.com/github/vijaysrajan/buysellconnect/blob/main/BasicGemma2bLocally.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Step 1: Install required packages with compatible versions
!pip install --upgrade transformers>=4.38.0
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade accelerate
!pip install langchain>=0.1.0
!pip install langchain-huggingface
!pip install huggingface_hub>=0.20.0


In [1]:

# Step 2: Restart runtime after installation (important!)
# Go to Runtime > Restart runtime in Colab, then run the cells below


In [2]:

# Step 3: Authenticate with Hugging Face (Gemma is a gated model)
from huggingface_hub import notebook_login
notebook_login()  # This will prompt you to enter your HF token


In [3]:

# Step 4: Import required libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.schema import HumanMessage


In [4]:

# Step 5: Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cpu


In [5]:

# Step 6: Load the model and tokenizer locally
model_id = "google/gemma-2b-it"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    low_cpu_mem_usage=True
)


Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:

# Step 7: Create a pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1000,
    do_sample=False,  # Greedy decoding
    return_full_text=False,
    device=0 if device == "cuda" else -1
)


Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [7]:

# Step 8: Create LangChain wrapper
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Step 9: Test the model
print("Testing the model...")
prompt = "Explain what is Generative AI in 3 bullet points"
response = llm.invoke(prompt)
print("Response:")
print(response)

# Alternative: Direct pipeline usage (without LangChain)
print("\n" + "="*50)
print("Alternative: Direct pipeline usage")
result = text_generation_pipeline(prompt)
print(result[0]['generated_text'])

Testing the model...
Response:
.

* Generative AI is a type of artificial intelligence that focuses on creating new content, such as images, videos, music, and text.
* It goes beyond traditional machine learning by focusing on the ability to generate novel and unexpected outputs.
* Generative AI models are trained on vast amounts of data and can produce content that is similar to the training data but also completely new.

Alternative: Direct pipeline usage
.

* Generative AI is a type of artificial intelligence that focuses on creating new content, such as images, videos, music, and text.
* It goes beyond traditional machine learning by focusing on the ability to generate novel and unexpected outputs.
* Generative AI models are trained on vast amounts of data and can produce content that is similar to the training data but also completely new.
